In [9]:
import math
import re
import pandas as pd
import tweepy
import emoji
import numpy as np
# from googleapiclient.discovery import build
from google.cloud import language
from google.cloud.language import types
from google.cloud.language import enums
# import matplotlib.pyplot as plt
from datetime import datetime

In [10]:
date = datetime.today().strftime('%Y-%m-%d')

# authorize tweepy
consumer_key = "OROBANkVOsvva9HETWL4Kovbx"
consumer_secret = "t87379Kk9ANccJDM5a6E6G5eLKDGLnL7s2zld1kXvUCYu9gjnJ"
access_token = "1305689531421786112-FNe1D2tjmvFjzikZYHDyjvRVAVT9gk"
access_token_secret = "G6KV4pfTg6l7BSXxbQZDyYbREmr2taNkYzBzSZZfDyBIa"

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit = True)

languages = ["ar","zh","nl","en","fr","de","id","it","ja","ko","pt","es","th","tr"]

client = language.LanguageServiceClient.from_service_account_json \
    ("/Users/phoenix/Documents/1stSemesterSO/DSCI125/Projects/Data-Science-Final-Project/DSCI_Auth.json")


In [11]:
def get_coordinates(filename):
    lat_long = pd.read_csv(filename,usecols = ["latitude","longitude","country"])
    lat_long["country"] = lat_long["country"].astype(str).apply(lambda s: s.strip())

    radius = pd.read_csv("countries of the world.csv",usecols = ["Country","Area (sq. mi.)"])
    radius["Country"] = radius["Country"].astype(str).apply(lambda s: s.strip())
    radius["Area (sq. mi.)"] = radius["Area (sq. mi.)"].astype(float).apply(lambda x: math.sqrt(x))
    radius.columns = ["country","area"]
    radius.columns = ["country","radius"]

    coordinates = pd.merge(lat_long,radius,on = ["country"])

    coordinates["lat-long-radius"] = coordinates["latitude"].astype(str) + "," + \
                                     coordinates["longitude"].astype(str) + \
                                     "," + coordinates["radius"].astype(str) + "mi"

    return coordinates

def read_csv_hfi(filename):
    df_byCountry = pd.read_csv(filename,usecols = ["countries","region","hf_score","hf_rank","hf_quartile",
                                                   "pf_expression","pf_association_assembly","pf_movement"],na_values = ["-"])

    df_byCountry.loc[:,"hf_score":"pf_movement"] = df_byCountry.loc[:,"hf_score":"pf_movement"].astype(float)

    aggregation_functions = {"region": "first","hf_score": "mean","hf_rank": "max","hf_quartile": "max",
                             "pf_expression": "mean","pf_association_assembly": "mean","pf_movement": "mean"}

    df_byRegion = df_byCountry.groupby(by = ["countries"]).aggregate(aggregation_functions)

    return df_byCountry,df_byRegion


# Twitter Interaction and Feature Extraction
# ---------------------------------------------------------------------------------------------------------------------
def filter_regex(text):
    text = re.sub(r"(?:\#|\@|https?\:(\/\/))\S+","",text)
    text = re.sub(r"\w*(RT|rt)\w*","",text)
    text = re.sub(r"(<[^>]+>)\S+","",text)

    text =  emoji.get_emoji_regexp().sub(u'',text)

    return text


# create get_tweets -> takes coordinates, a datetime range to search, result type, maximum number of tweets to scrape
def get_tweets(coordinates,result_type,until_date,count,max_tweets):
    """Takes in 5 parameters, set the max_tweets to 150, count to 25, and result type to 'recent'
    until date as the current date, and the coordinates equal to the 'latitude' and 'longitude'
    values. Returns dataframe with following columns -> text, created_at_date, favorite_count,
    user location, followers count, friends count, and the language used.
    Indexed 0 - 6"""
    tweets = tweepy.Cursor(api.search,geocode = coordinates,result_type = result_type,
                           until = until_date,count = count).items(max_tweets)

    tweets_list = [
         [filter_regex(tweet.text),tweet.created_at,tweet.retweet_count,tweet.favorite_count,
          tweet.user.location,tweet.user.followers_count,tweet.user.friends_count,
          tweet.lang] for tweet in tweets]


    tweets_df = pd.DataFrame(data = tweets_list,columns = ["text","created_at","retweet_count","favorite_count",
                                                           "user_location","followers_count",
                                                           "friends_count","language"])

    return tweets_df


def get_tweets_by_country(coordinates_df):
    """Takes in tweet_df, run the get_tweets to get all those tweets
    Uses latitude, longitude, and radius, return a dataframe with each iteration
    Save into list of dataframes, and merge the list into a single frame
    byCountry -> country, all the columns of tweets_df"""
    # return the dataframe with country, lat, and longitude, radius
    result_type = 'recent'
    max_tweets = 500

    tweets_by_country = pd.DataFrame()

    print(f"retrieving tweets, 5000 tweets remaining")

    # go through the dataframe for the coordinates by country, and retrieve the tweets from each country (~25)
    # save results into dataframe with following characteristics -> country, text
    for coordinate in coordinates_df.itertuples():
        tweets = get_tweets(coordinate[2],result_type,date,150,max_tweets)
        tweets["region"] = coordinate[1]
        tweets_by_country = pd.concat([tweets_by_country,tweets])
        print(f"retrieving tweets, {5000 - len(tweets_by_country)} tweets remaining")

    tweets_by_country = tweets_by_country[tweets_by_country["language"].isin(languages)]

    return tweets_by_country

In [12]:
# df,df_aggregates = read_csv_hfi("hfi_cc_2019.csv")
# df_by_country,df_by_region = read_csv_hfi("hfi_cc_2019.csv")
# df_coordinates = get_coordinates("lat_long_coordinates.csv")

# df_regional_coord = pd.read_csv("regional_coordinates .csv")

# df_tweets = get_tweets_by_country(df_regional_coord)

In [13]:
df_tweets

,text,created_at,retweet_count,favorite_count,user_location,followers_count,friends_count,language,region
0,مين كان يدري إن الاغنية الي كنت احبها وانا صغي...,2020-11-28 23:59:59,0,0,saudi Arabia,105,298,ar,Caucasus & Central Asia
1,\nنوفمبر شهر النسيم البارد\n وبدايات الشتاء...,2020-11-28 23:59:59,125,0,,915,1066,ar,Caucasus & Central Asia
2,فاهم انك ناقل وانا بكلامي اقصده هو,2020-11-28 23:59:59,0,0,بريدة,8,56,ar,Caucasus & Central Asia
3,بطلاً لكأس خادم الحرمين الشريفين \n\nمبروو...,2020-11-28 23:59:59,27915,0,,263,36,ar,Caucasus & Central Asia
4,ععلييك اماني,2020-11-28 23:59:59,0,0,Jeddah🌴,15470,8060,ar,Caucasus & Central Asia
...,...,...,...,...,...,...,...,...,...
492,Cool! That Chinese dragon on your back it som...,2020-11-28 23:59:49,0,0,"Rotterdam, Nederland",489,650,en,Western Europe
493,Ce que vit la police aujourd’hui est tout au...,2020-11-28 23:59:49,5745,0,"Montgermont, France",192,292,fr,Western Europe
494,"""Bande de PD"" prononcé par un flic.\nMais ou...",2020-11-28 23:59:49,167,0,"Grenoble, France",389,270,fr,Western Europe
497,Well work changed we dont work like you do,2020-11-28 23:59:49,0,0,Sneek,387,2053,en,Western Europe


In [117]:
def find_sentiment_score(tweets_text):# ,tweets_lang):

    tweets = pd.DataFrame(columns = ["score","magnitude"])

    document = types.Document(
        content = tweets_text,
        # language = tweets_lang,
        type = enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    response = client.analyze_sentiment(document = document,encoding_type = "UTF8").document_sentiment.score
    
    return response


def find_sentiment_magnitude(tweets_text):# ,tweets_lang):

    tweets = pd.DataFrame(columns = ["score","magnitude"])

    document = types.Document(
        content = tweets_text,
        # language = tweets_lang,
        type = enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    response = client.analyze_sentiment(document = document,encoding_type = "UTF8").document_sentiment.magnitude
    
    return response

In [17]:
df_tweets = get_tweets_by_country(df_regional_coord)

retrieving tweets, 5000 tweets remaining
retrieving tweets, 4500 tweets remaining
retrieving tweets, 4000 tweets remaining
retrieving tweets, 3500 tweets remaining
retrieving tweets, 3000 tweets remaining
retrieving tweets, 2500 tweets remaining
retrieving tweets, 2000 tweets remaining
retrieving tweets, 1500 tweets remaining
retrieving tweets, 1000 tweets remaining
retrieving tweets, 500 tweets remaining
retrieving tweets, 0 tweets remaining


In [118]:
# df_sentiment = df_tweets

# df_sentiment["score"] = df_sentiment.text.apply(find_sentiment_score)
# df_sentiment["magnitude"] = df_sentiment.text.apply(find_sentiment_magnitude)
def build_tweet_dataset(tweets):
    df,df_aggregates = read_csv_hfi("hfi_cc_2019.csv")
    df_by_country,df_by_region = read_csv_hfi("hfi_cc_2019.csv")
    df_coordinates = get_coordinates("lat_long_coordinates.csv")

    df_regional_coord = pd.read_csv("regional_coordinates .csv")

    

    regions = ['Caucasus & Central Asia', 'East Asia', 'Eastern Europe',
     'Latin America & the Caribbean', 'Middle East & North Africa',
     'North America', 'Oceania', 'South Asia', 'Sub-Saharan Africa',
     'Western Europe']
    
    by_region_dataframe = pd.DataFrame()
    
    tracker = 0
    for r in regions:
        tweet = tweets.loc[tweets["region"] == r].iloc[:100] 
        by_region_dataframe = pd.concat([by_region_dataframe,tweet])
    
    return by_region_dataframe

In [39]:
tweets = build_tweet_dataset(df_tweets)

In [119]:
# drop row 83,460
def find_sentiment_main(tweets):
    # tweets = build_tweet_dataset()
    
    df_sentiment = tweets.reset_index()

    df_sentiment = df_sentiment.drop([83,460])
    
    df_sentiment["score"] = df_sentiment.text.apply(find_sentiment_score)
    df_sentiment["magnitude"] = df_sentiment.text.apply(find_sentiment_magnitude)

    return df_sentiment


df = find_sentiment_main(tweets)
df

,index,text,created_at,retweet_count,favorite_count,user_location,followers_count,friends_count,language,region,score,magnitude
0,7,اول مرة في حياتي اتفق مع اعلام النصر ....!,2020-11-28 23:59:59,818,0,,260,963,ar,Caucasus & Central Asia,0.6,0.6
1,221,الله يبارك فيك يارب,2020-11-28 23:59:53,0,0,Kingdom of Saudi Arabia,56,97,ar,Caucasus & Central Asia,0.9,0.9
2,315,what could go wrong? ¯\_(ツ)_/¯,2020-11-28 23:59:51,0,1,"the burg, PA",14,54,en,Caucasus & Central Asia,0.3,0.7
3,226,cem karacanın sıkıldığına yemin edebilirim am...,2020-11-28 23:59:53,0,3,"Mersin, Türkiye",1741,2346,tr,Caucasus & Central Asia,0.3,0.3
4,443,Yatsam burda mı olurdum hchxjxkc,2020-11-28 23:59:47,0,0,"Konya, Türkiye",10767,9073,tr,Caucasus & Central Asia,-0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,34,my peace is my top priority rn,2020-11-28 23:59:59,12472,0,🇵🇹,635,631,en,Western Europe,0.9,0.9
996,427,derrière il est en foule panique elle elle s...,2020-11-28 23:59:50,11,0,,638,202,fr,Western Europe,-0.6,0.6
997,261,been trippin bout some things can’t change,2020-11-28 23:59:53,0,1,"Ulm, Germany",57,62,en,Western Europe,-0.8,0.8
998,176,Hatem Ben Arfa avec le en \n-23 matchs disp...,2020-11-28 23:59:55,9,0,ANGERS,414,140,fr,Western Europe,0.1,0.3


In [120]:
df.to_csv("sentiment_analysis_byRegion.csv")